In [10]:
import os
import openai
 
# openai.api_key = os.environ["OPENAI_API_KEY"]
# openai.api_base_url = os.environ["OPENAI_API_URL"]

openai.api_key = "ak-rijxawD5sZxxnXXEoJmpWG52QlAMYxxaEQmp1ixrcJQOtbVY"
openai.api_base_url = "https://api.nextapi.fun/v1"

# def llm(prompt, stop=["\n"]):
#     response = openai.Completion.create(
#       model="text-davinci-002",
#       prompt=prompt,
#       temperature=0,
#       max_tokens=100,
#       top_p=1,
#       frequency_penalty=0.0,
#       presence_penalty=0.0,
#       stop=stop
#     )
#     return response["choices"][0]["text"]

from openai import OpenAI
client = OpenAI(
    api_key = openai.api_key,
    base_url= openai.base_url,
)

def llm(prompt, model="gpt-3.5-turbo-0125", stop=None):
    if stop is None:
        stop = ["\n"]
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0,
        stop=stop,
    )
    return response.choices[0].message.content


In [2]:
import yaml
import alfworld
import alfworld.agents.environment
with open('base_config.yaml') as reader:
    config = yaml.safe_load(reader)
    
split = "eval_out_of_distribution"

env = getattr(alfworld.agents.environment, config["env"]["type"])(config, train_eval=split)
env = env.init_env(batch_size=1)

def process_ob(ob):
    if ob.startswith('You arrive at loc '):
        ob = ob[ob.find('. ')+2:]    
    return ob

Initializing AlfredTWEnv...


100%|██████████| 341/341 [00:00<00:00, 1106.67it/s]

Overall we have 134 games in split=eval_out_of_distribution
Evaluating with 134 games


In [4]:
import json
folder = './data/prompts/'
prompt_file = 'alfworld_3prompts.json'
with open(folder + prompt_file, 'r') as f:
    d = json.load(f)

In [5]:
import sys

def alfworld_run(prompt, to_print=True, ob=''):
    init_prompt = prompt + ob + '\n>'
    prompt = ''
    if to_print:
        print(ob)
        sys.stdout.flush()
    for i in range(1, 50):
        action = llm(init_prompt + prompt, stop=['\n']).strip()
        observation, reward, done, info = env.step([action])
        observation, reward, done = process_ob(observation[0]), info['won'][0], done[0]
        if action.startswith('think:'):
            observation = 'OK.'
        if to_print:
            print(f'Act {i}: {action}\nObs {i}: {observation}')
            sys.stdout.flush()
        prompt += f' {action}\n{observation}\n>'
        if done:
            return reward
    return 0

In [11]:
prefixes = {
    'pick_and_place': 'put',
    'pick_clean_then_place': 'clean',
    'pick_heat_then_place': 'heat',
    'pick_cool_then_place': 'cool',
    'look_at_obj': 'examine',
    'pick_two_obj': 'puttwo'
}
cnts = [0] * 6  #总数，一共6个任务
rs = [0] * 6    #结果，一共6个任务

for _ in range(134):
    ob, info = env.reset()  # info 中包含了 won (list，布尔值）、'admissible_commands'（二级list，可行指令）、任务文件信息
    ob = '\n'.join(ob[0].split('\n\n')[1:]) # ob是任务描述，最后一行格式是 "Your task is to: put a clean spatula in drawer." 
    name = '/'.join(info['extra.gamefile'][0].split('/')[-3:-1]) # 获得任务名前缀，判断是什么类型任务
    print(name)
    for i, (k, v) in enumerate(prefixes.items()):
        if name.startswith(k):
            prompt = 'Interact with a household to solve a task. Here are two examples.\n' + d[f'react_{v}_1'] + d[f'react_{v}_0'] + '\nHere is the task.\n'
            print(k, v)
            r = alfworld_run(prompt, ob=ob)
            rs[i] += r
            cnts[i] += 1
            break
    print(_+1, 'r', r, 'rs', rs, 'cnts', cnts, 'sum(rs)/sum(cnts)', sum(rs) / sum(cnts))
    print('------------\n')


pick_clean_then_place_in_recep-Cloth-None-Cabinet-424/trial_T20190908_022436_073995
pick_clean_then_place clean
You are in the middle of a room. Looking quickly around you, you see a cabinet 4, a cabinet 3, a cabinet 2, a cabinet 1, a countertop 1, a garbagecan 1, a handtowelholder 2, a handtowelholder 1, a sinkbasin 2, a sinkbasin 1, a toilet 1, a toiletpaperhanger 1, and a towelholder 1.
Your task is to: clean some cloth and put it in cabinet.


AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: ak-rijxa***************************************tbVY. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}